In [ ]:
%cd /kaggle/working/
!rm -r climategan/
! git clone https://github.com/ManhHoDinh/climategan.git
%cd climategan
!pip install -r requirements-3.8.2.txt # or `requirements-any.txt` for other Python versions (not tested but expected to be fine)


!pip install gdown
!mkdir config
!mkdir output
%cd config
!gdown https://drive.google.com/u/0/uc?id=18OCUIy7JQ2Ow_-cC5xn_hhDn-Bp45N1K
!unzip release-github-v1.zip
%cd /kaggle/working/

# Install detectron2 (colab has CUDA 10.1 + torch 1.5)
import numpy as np
import os, json, cv2, random

import os
HOME = os.getcwd()
print("HOME:", HOME)

!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

%cd ..
mask_generator = SamAutomaticMaskGenerator(sam)

import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
mask_predictor = SamPredictor(sam)

# helper function that loads an image before adding it to the widget

import base64

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Union, Optional
from dataclasses_json import dataclass_json
from supervision import Detections


@dataclass_json
@dataclass
class COCOCategory:
    id: int
    name: str
    supercategory: str


@dataclass_json
@dataclass
class COCOImage:
    id: int
    width: int
    height: int
    file_name: str
    license: int
    date_captured: str
    coco_url: Optional[str] = None
    flickr_url: Optional[str] = None


@dataclass_json
@dataclass
class COCOAnnotation:
    id: int
    image_id: int
    category_id: int
    segmentation: List[List[float]]
    area: float
    bbox: Tuple[float, float, float, float]
    iscrowd: int


@dataclass_json
@dataclass
class COCOLicense:
    id: int
    name: str
    url: str


@dataclass_json
@dataclass
class COCOJson:
    images: List[COCOImage]
    annotations: List[COCOAnnotation]
    categories: List[COCOCategory]
    licenses: List[COCOLicense]


def load_coco_json(json_file: str) -> COCOJson:
    import json

    with open(json_file, "r") as f:
        json_data = json.load(f)

    return COCOJson.from_dict(json_data)


class COCOJsonUtility:
    @staticmethod
    def get_annotations_by_image_id(coco_data: COCOJson, image_id: int) -> List[COCOAnnotation]:
        return [annotation for annotation in coco_data.annotations if annotation.image_id == image_id]

    @staticmethod
    def get_annotations_by_image_path(coco_data: COCOJson, image_path: str) -> Optional[List[COCOAnnotation]]:
        image = COCOJsonUtility.get_image_by_path(coco_data, image_path)
        if image:
            return COCOJsonUtility.get_annotations_by_image_id(coco_data, image.id)
        else:
            return None

    @staticmethod
    def get_image_by_path(coco_data: COCOJson, image_path: str) -> Optional[COCOImage]:
        for image in coco_data.images:
            if image.file_name == image_path:
                return image
        return None

    @staticmethod
    def annotations2detections(annotations: List[COCOAnnotation]) -> Detections:
        class_id, xyxy = [], []

        for annotation in annotations:
            x_min, y_min, width, height = annotation.bbox
            class_id.append(annotation.category_id)
            xyxy.append([
                x_min,
                y_min,
                x_min + width,
                y_min + height
            ])

        return Detections(
            xyxy=np.array(xyxy, dtype=int),
            class_id=np.array(class_id, dtype=int)
        )
!pip install roboflow
import cv2
import numpy as np

def resize_and_overlay_images(foreground, background):
    # Resize foreground to match the size of the background
    foreground_resized = cv2.resize(foreground, (background.shape[1], background.shape[0]), interpolation=cv2.INTER_AREA)

    # Separate the channels of the foreground image
    foreground_resized_rgb = foreground_resized[:, :, :3]  # RGB channels
    alpha_channel = foreground_resized[:, :, 3]  # Alpha channel

    # Normalize the alpha mask to keep intensity between 0 and 1
    alpha_mask = alpha_channel.astype(float) / 255

    # Make alpha_mask 3 channels
    alpha_mask_3d = np.stack([alpha_mask, alpha_mask, alpha_mask], axis=-1)

    # Multiply the foreground with the alpha matte
    foreground_resized_rgb = cv2.multiply(alpha_mask_3d, foreground_resized_rgb.astype(float))

    # Multiply the background with ( 1 - alpha )
    background_rgb = background[:, :, :3]  # Get the RGB (assuming the fourth channel is alpha)
    alpha_mask_inv = 1 - alpha_mask_3d
    background_rgb = cv2.multiply(alpha_mask_inv, background_rgb.astype(float))

    # Add the foreground and background.
    combined_rgb = cv2.add(foreground_resized_rgb, background_rgb)

    # Return the combined image
    combined_image = combined_rgb.astype(np.uint8)

    return combined_image
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

def insertImageToImage(foreground, originImage):
    foreground=resize_and_overlay_images(foreground, originImage)
    # Convert the OpenCV images to PIL format, assuming the OpenCV images are in BGR format.
    foreground_pil = Image.fromarray(cv2.cvtColor(foreground, cv2.COLOR_BGRA2RGBA))
    background_pil = Image.fromarray(cv2.cvtColor(originImage, cv2.COLOR_BGR2RGB))

    # Since the images are the same size, we can paste the foreground on the background directly.
    background_pil.paste(foreground_pil, (0, 0), foreground_pil)

    # Convert back to OpenCV format
    result = cv2.cvtColor(np.array(background_pil), cv2.COLOR_RGB2BGR)
    return result
!pip install roboflow